In [45]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import io
import os
from tabulate import tabulate
from typing import Literal, List
import re
sns.set_style("whitegrid")

first load data, get all years together
what is devtest???

ik doe wat ze in paper hebben gedaan (alle data van vorige jaren = train,+ train van dit jaar), maar toch is er evenveel test als train ongeveer, wat ik raar vind.) het lijkt alsof er minder train dan test data is ook (bijv 2015) -> dat is toch raar hahah

In [54]:
def get_filenames(dataset: Literal["train","test"], path="data/downloaded/") -> List[str]:
    if dataset == "train":
        r = '[a-z]+-(\d{4})'
        onlyfiles = [f for f in os.listdir("data/downloaded") if os.path.isfile(os.path.join("data/downloaded", f)) and os.path.join("data/downloaded", f)[-5] == "A"]
        return [f for f in onlyfiles if f not in ["twitter-2016dev-A.tsv", "twitter-2016devtest-A.tsv", "twitter-2016train-A.tsv"]]
    return ["twitter-2016dev-A.tsv", "twitter-2016devtest-A.tsv", "twitter-2016train-A.tsv"]

In [103]:
def concatenate_data(dataset: Literal["train","test"], path="data/downloaded/") -> None:
    # write all data from previous years to new file, as in paper
    filenames = get_filenames(dataset, path=path)

    with open(f'{path}twitter-{dataset}.tsv','w', encoding="utf-8") as outfile:
        for fname in filenames:
            with open(path+fname, "r",encoding="utf-8") as infile:
                for line in infile:
                    outfile.write(line)

In [97]:
concatenate_data("train")
concatenate_data("test")

Load into dataframes

In [121]:
train = pd.read_csv("data/downloaded/twitter-train.tsv", sep='\t', usecols=[1,2], names=["label", "text"])
test = pd.read_csv("data/downloaded/twitter-test.tsv", sep='\t', usecols=[1,2], names=["label", "text"])
# dev = pd.read_csv("data/downloaded/twitter-2016dev-A.tsv", sep='\t', usecols=[1,2], names=["label", "text"])

print(f"train: {round(train.shape[0]/(train.shape[0]+test.shape[0])*100)}%, test: {round(test.shape[0]/(train.shape[0]+test.shape[0])*100)}%")
print(f"train: {train.shape[0]}, test: {test.shape[0]}")

train: 80%, test: 20%
train: 40297, test: 9834


In [122]:
# train.text = train.text.astype('unicode')
train.text[20632].encode("ascii")

b'Theo Walcott is still shit\\u002c watch Rafa and Johnny deal with him on Saturday.'

class imbalances

In [61]:
train_class_counts = train.groupby('label').count().rename({"text":"train"}, axis=1)
test_class_counts = test.groupby('label').count().rename({"text":"test"}, axis=1)
# dev_class_counts = dev.groupby('label').count().rename({"text":"dev"}, axis=1)

#TODO we moeten nog dev hebben: 80/20 split van tex


pd.concat([train_class_counts, test_class_counts], axis=1)

,train,test
label,,
negative,6243,1566
neutral,19096,3428
positive,14958,4840


preprocessing:
remove links? 
remove @'s ?
punctuation etc? 


In [62]:
train.head()

,label,text
0,neutral,"Picturehouse's, Pink Floyd's, 'Roger Waters: T..."
1,neutral,Order Go Set a Watchman in store or through ou...
2,negative,If these runway renovations at the airport pre...
3,neutral,If you could ask an onstage interview question...
4,positive,A portion of book sales from our Harper Lee/Go...


In [137]:
# preprocessing:

def preprocess(data: pd.DataFrame, name: str) -> pd.DataFrame:
        
    # remove urls
    data_clean = data.copy()
    data_clean['text']=data_clean.text.apply(lambda x: re.sub(r'https?:\/\/\S+','',x))

    # remove emailadresses
    data_clean['text']=data_clean.text.apply(lambda x: re.sub(r'\w+@\w+\.[a-z]+','',x))

    # remove html ref
    data_clean['text']=data_clean.text.apply(lambda x: re.sub(r'&[a-z]+;', '', x))

    data_clean['text']=data_clean.text.apply(lambda x: re.sub(r'&amp', '&', x))

    # remove unicode
    data_clean['text']=data_clean.text.apply(lambda x: re.sub(r'\\u[a-z0-9]{4}', '', x))

    # remove all @ mention
    data_clean['text']=data_clean.text.apply(lambda x: re.sub(r'@\w+', '', x))

    # remove non-letter characters, numbers 
    data_clean['text']=data_clean.text.apply(lambda x: re.sub(r"[^a-zA-Z\s\:\];='\.\!\?\,]", ' ', x))
    data_clean['text']=data_clean.text.apply(lambda x: re.sub(r"([.,!?\s])st([.,!?\s])", ' ', x))
    data_clean['text']=data_clean.text.apply(lambda x: re.sub(r"([.,!?\s])th([.,!?\s])", ' ', x))
    data_clean['text']=data_clean.text.apply(lambda x: re.sub(r"([.,!?\s])nd([.,!?\s])", ' ', x))
    data_clean['text']=data_clean.text.apply(lambda x: re.sub(r"([.,!?\s])rd([.,!?\s])", ' ', x))


    # remove trailing whitespace
    data_clean['text']=data_clean.text.apply(lambda x: re.sub(r'\s+', ' ', x).strip())

    data_clean.to_csv(f"data/downloaded/twitter-{name}-clean.tsv", sep="\t", index=False,header=False,encoding="utf-8")

    return data_clean
# overige dingen: afkortingen? getallen kunnen belangrijk zijn

In [136]:
train_cleaned = preprocess(train, "train")
test_cleaned = preprocess(test, "test")

In [ ]:
# experiment: probeer de verschillende instellingen aangeraden door autheurs